In [1]:
import pandas as pd
import torch
import pprint
import evaluate
import numpy as np
 

In [2]:
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)

In [3]:
dataset = pd.read_csv('data/processed/processed_data.csv')

In [7]:
dataset1 = dataset.head(1000)


In [8]:
from datasets import Dataset
dataset1 = Dataset.from_pandas(dataset1)

In [9]:
full_dataset = dataset1.train_test_split(test_size=0.2, shuffle=True)
dataset_train = full_dataset['train']
dataset_valid = full_dataset['test']

print(dataset_train)
print(dataset_valid)

Dataset({
    features: ['Summary', 'Text'],
    num_rows: 800
})
Dataset({
    features: ['Summary', 'Text'],
    num_rows: 200
})


In [10]:
MODEL = 't5-small'
BATCH_SIZE = 4
NUM_PROCS = 4
EPOCHS = 5
OUT_DIR = './result666'
MAX_LENGTH = 512 

In [11]:
tokenizer = T5Tokenizer('models/trained_spiece.model')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [12]:
from functools import partial
from transformers import T5Tokenizer

# Initialize the tokenizer
# tokenizer = T5Tokenizer.from_pretrained(MODEL)
tokenizer = T5Tokenizer('models/trained_spiece.model')
# Function to convert text data into model inputs and targets
def preprocess_function(examples, tokenizer):
    inputs = [f"summarize: {text}" for text in examples['Text']]
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding='max_length'
    )

    # Set up the tokenizer for targets
    targets = [summary for summary in examples['Summary']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=512,
            truncation=True,
            padding='max_length'
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Create a partial function with the tokenizer
preprocess_function_with_tokenizer = partial(preprocess_function, tokenizer=tokenizer)

# Apply the function to the whole dataset
tokenized_train = dataset_train.map(
    preprocess_function_with_tokenizer,
    batched=True,
    num_proc=NUM_PROCS
)
tokenized_valid = dataset_valid.map(
    preprocess_function_with_tokenizer,
    batched=True,
    num_proc=NUM_PROCS
)


Map (num_proc=4):   0%|          | 0/800 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [14]:
model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

60,506,624 total parameters.
60,506,624 training parameters.


In [15]:
rouge = evaluate.load("rouge")
 
def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids
 
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
 
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=[
            'rouge1',
            'rouge2',
            'rougeL'
        ]
    )
 
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
 
    return {k: round(v, 4) for k, v in result.items()}

In [16]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [17]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=OUT_DIR,
    logging_steps=10,
    eval_strategy='steps',
    eval_steps=200,
    save_strategy='epoch',
    save_total_limit=2,
    report_to='tensorboard',
    learning_rate=0.0001,
    dataloader_num_workers=4
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)
 
history = trainer.train()

  0%|          | 0/1000 [00:00<?, ?it/s]

{'loss': 19.1766, 'grad_norm': 143.90223693847656, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.05}
{'loss': 17.9078, 'grad_norm': 139.80279541015625, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.1}
{'loss': 17.4335, 'grad_norm': 184.06051635742188, 'learning_rate': 6e-06, 'epoch': 0.15}
{'loss': 15.7727, 'grad_norm': 108.15489196777344, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.2}
{'loss': 14.0783, 'grad_norm': 109.02210235595703, 'learning_rate': 1e-05, 'epoch': 0.25}
{'loss': 12.5055, 'grad_norm': 98.36102294921875, 'learning_rate': 1.2e-05, 'epoch': 0.3}
{'loss': 10.332, 'grad_norm': 93.169921875, 'learning_rate': 1.4000000000000001e-05, 'epoch': 0.35}
{'loss': 8.4961, 'grad_norm': 830.5044555664062, 'learning_rate': 1.6000000000000003e-05, 'epoch': 0.4}
{'loss': 5.3842, 'grad_norm': 72.83383178710938, 'learning_rate': 1.8e-05, 'epoch': 0.45}
{'loss': 3.2576, 'grad_norm': 60.221187591552734, 'learning_rate': 2e-05, 'epoch': 0.5}
{'loss': 1.9472, 'grad_norm': 1

  0%|          | 0/50 [00:00<?, ?it/s]

{'eval_loss': 0.5537384748458862, 'eval_rouge1': 0.4595, 'eval_rouge2': 0.1798, 'eval_rougeL': 0.3448, 'eval_gen_len': 38.305, 'eval_runtime': 20.832, 'eval_samples_per_second': 9.601, 'eval_steps_per_second': 2.4, 'epoch': 1.0}
{'loss': 0.5991, 'grad_norm': 0.38639014959335327, 'learning_rate': 4.2e-05, 'epoch': 1.05}
{'loss': 0.5688, 'grad_norm': 0.5554342269897461, 'learning_rate': 4.4000000000000006e-05, 'epoch': 1.1}
{'loss': 0.5192, 'grad_norm': 0.42708340287208557, 'learning_rate': 4.600000000000001e-05, 'epoch': 1.15}
{'loss': 0.531, 'grad_norm': 0.4686841368675232, 'learning_rate': 4.8e-05, 'epoch': 1.2}
{'loss': 0.5264, 'grad_norm': 0.645159125328064, 'learning_rate': 5e-05, 'epoch': 1.25}
{'loss': 0.5305, 'grad_norm': 0.6609600782394409, 'learning_rate': 5.2000000000000004e-05, 'epoch': 1.3}
{'loss': 0.5193, 'grad_norm': 0.3591311573982239, 'learning_rate': 5.4000000000000005e-05, 'epoch': 1.35}
{'loss': 0.5415, 'grad_norm': 0.5596582293510437, 'learning_rate': 5.60000000000

  0%|          | 0/50 [00:00<?, ?it/s]

...

{'eval_loss': 0.47073325514793396, 'eval_rouge1': 0.5351, 'eval_rouge2': 0.2144, 'eval_rougeL': 0.3876, 'eval_gen_len': 39.585, 'eval_runtime': 20.9352, 'eval_samples_per_second': 9.553, 'eval_steps_per_second': 2.388, 'epoch': 3.0}
{'loss': 0.4531, 'grad_norm': 0.28173404932022095, 'learning_rate': 7.800000000000001e-05, 'epoch': 3.05}
{'loss': 0.4364, 'grad_norm': 0.285885751247406, 'learning_rate': 7.6e-05, 'epoch': 3.1}
{'loss': 0.4398, 'grad_norm': 0.3147886395454407, 'learning_rate': 7.4e-05, 'epoch': 3.15}
{'loss': 0.473, 'grad_norm': 0.47332173585891724, 'learning_rate': 7.2e-05, 'epoch': 3.2}
{'loss': 0.4622, 'grad_norm': 0.3438221216201782, 'learning_rate': 7e-05, 'epoch': 3.25}
{'loss': 0.4199, 'grad_norm': 0.3423343598842621, 'learning_rate': 6.800000000000001e-05, 'epoch': 3.3}
{'loss': 0.502, 'grad_norm': 0.33004891872406006, 'learning_rate': 6.6e-05, 'epoch': 3.35}
{'loss': 0.4771, 'grad_norm': 0.336498498916626, 'learning_rate': 6.400000000000001e-05, 'epoch': 3.4}
{'lo

  0%|          | 0/50 [00:00<?, ?it/s]

{'eval_loss': 0.47123128175735474, 'eval_rouge1': 0.552, 'eval_rouge2': 0.2215, 'eval_rougeL': 0.3969, 'eval_gen_len': 39.635, 'eval_runtime': 21.3552, 'eval_samples_per_second': 9.365, 'eval_steps_per_second': 2.341, 'epoch': 4.0}
{'loss': 0.4906, 'grad_norm': 0.2612118124961853, 'learning_rate': 3.8e-05, 'epoch': 4.05}
{'loss': 0.4372, 'grad_norm': 0.3074450194835663, 'learning_rate': 3.6e-05, 'epoch': 4.1}
{'loss': 0.4622, 'grad_norm': 0.3396832346916199, 'learning_rate': 3.4000000000000007e-05, 'epoch': 4.15}
{'loss': 0.4554, 'grad_norm': 0.2993573844432831, 'learning_rate': 3.2000000000000005e-05, 'epoch': 4.2}
{'loss': 0.4571, 'grad_norm': 0.4288477599620819, 'learning_rate': 3e-05, 'epoch': 4.25}
{'loss': 0.4687, 'grad_norm': 0.2666241526603699, 'learning_rate': 2.8000000000000003e-05, 'epoch': 4.3}
{'loss': 0.4192, 'grad_norm': 0.41558536887168884, 'learning_rate': 2.6000000000000002e-05, 'epoch': 4.35}
{'loss': 0.4995, 'grad_norm': 0.38977375626564026, 'learning_rate': 2.4e-05

  0%|          | 0/50 [00:00<?, ?it/s]

{'eval_loss': 0.46943679451942444, 'eval_rouge1': 0.5524, 'eval_rouge2': 0.2214, 'eval_rougeL': 0.3955, 'eval_gen_len': 39.665, 'eval_runtime': 20.8228, 'eval_samples_per_second': 9.605, 'eval_steps_per_second': 2.401, 'epoch': 5.0}
{'train_runtime': 377.3843, 'train_samples_per_second': 10.599, 'train_steps_per_second': 2.65, 'train_loss': 1.7211099138259889, 'epoch': 5.0}


In [18]:
model_path = f"{OUT_DIR}/checkpoint-1000"  # the path where you saved your model
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer('models/trained_spiece.model')

In [64]:
def summarize_text(text, model, tokenizer, max_length=512, num_beams=5):
    # Preprocess the text
    inputs = tokenizer.encode(
        "summarize: " + text,
        return_tensors='pt',
        max_length=max_length,
        truncation=True
    )

    # Generate the summary
    summary_ids = model.generate(
        inputs,
        max_length=200,
        num_beams=num_beams,
        # early_stopping=True,
    )

    # Decode and return the summary
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [61]:
text = 'Khu vực Nam Bộ có nắng nóng và nắng nóng gay gắt trong các ngày từ 1-8/5; sau đó từ 9/5, nắng nóng chủ yếu tập trung tại các tỉnh miền Đông Nam Bộ. Trên cả nước đã ghi nhận nhiều trạm khí tượng xảy ra giá trị nhiệt độ cao nhất ngày vượt giá trị lịch sử. Cụ thể, cùng trong ngày 1/5, ở Đông Hà (Quảng Trị) đạt đến 43,2 độ, vượt mức 42,3 độ năm 2023; Huế (Thừa Thiên Huế) 42,1 độ, vượt kỷ lục 41,3 độ năm 1983; Đà Nẵng 41,5, vượt mức 40,5 năm 1983 hay Thủ Dầu Một (Bình Dương) 38,9 độ ngày 2/5, vượt mức 38,7 năm 2016,…'
summary = dataset['Summary'][50000]
print(text)

Khu vực Nam Bộ có nắng nóng và nắng nóng gay gắt trong các ngày từ 1-8/5; sau đó từ 9/5, nắng nóng chủ yếu tập trung tại các tỉnh miền Đông Nam Bộ. Trên cả nước đã ghi nhận nhiều trạm khí tượng xảy ra giá trị nhiệt độ cao nhất ngày vượt giá trị lịch sử. Cụ thể, cùng trong ngày 1/5, ở Đông Hà (Quảng Trị) đạt đến 43,2 độ, vượt mức 42,3 độ năm 2023; Huế (Thừa Thiên Huế) 42,1 độ, vượt kỷ lục 41,3 độ năm 1983; Đà Nẵng 41,5, vượt mức 40,5 năm 1983 hay Thủ Dầu Một (Bình Dương) 38,9 độ ngày 2/5, vượt mức 38,7 năm 2016,…


In [58]:
print(summary)

Kinh tế Trung Quốc tăng tốc trong khi các tin tức về vaccine không đủ kéo châu Âu và Mỹ phục hồi nhanh hơn.


In [65]:
summarize_text(text, model, tokenizer)

'nắng nóng chủ yếu tập trung tại các tỉnh miên Đông Nam Bộ. Trên cả nước đã ghi nhận nhiêu trạm khí tượng xảy ra giá trị nhiệt độ cao nhất ngày từ 1-8/5; sau đó từ 9/5, nắng nóng chủ yếu tập trung tại các tỉnh miên Đông Nam Bộ.'

In [73]:
history.metrics

{'train_runtime': 377.3843,
 'train_samples_per_second': 10.599,
 'train_steps_per_second': 2.65,
 'total_flos': 541367205888000.0,
 'train_loss': 1.7211099138259889,
 'epoch': 5.0}